In [ ]:
#import os
#from google.colab import drive

#drive.mount('/content/drive')
#os.chdir('/content/drive/My Drive/siamese_netowrk_dissimilarity_calculation')


#import sys
#sys.path.append('/content/drive/My Drive/siamese_netowrk_dissimilarity_calculation')



In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import cv2 
import re
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers


In [ ]:
# create dictionary of lichens 

lichens = os.listdir('../data/train')
lichens_dict = {}
for i,l in enumerate(lichens):
    lichens_dict[l] = i 

In [ ]:
os.listdir('../')

PRIMA COSA: gestire i dati e le immagini delle patch

In [ ]:
im = cv2.imread('../data/train/Arthonia_radiata/Arthonia_radiata_texture01_crop_ 1.jpg')
im = cv2.resize(im, (100,100), interpolation = cv2.INTER_AREA)
plt.imshow(im)

In [ ]:
 


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
def create_lichen_dataframe(pth, lichens_dict):
    """
    l'obiettivo di questa funzione è quello di creare un dataframe Pandas che abbia le seguenti colonne : FILEPATH, LABEL, LABEL_ID
    """

    lista_licheni = os.listdir(pth)
    lista_totale_licheni  = []
    for i,lich in enumerate(lista_licheni):
        path_singolo_lichene = os.path.join(pth,lich)
        immagini_singolo_lichene = os.listdir(path_singolo_lichene)
        for j,img in enumerate(immagini_singolo_lichene):
            # salvo immagine_pth, label. e label_id
            filepath = os.path.join(path_singolo_lichene, img)
            label = lich 
            label_idx = lichens_dict[lich]
            lista_totale_licheni.append([filepath, label, label_idx])


    res = pd.DataFrame(data = lista_totale_licheni, columns = ['filepath','class','label'], index = None)  
    return res




In [ ]:
def create_dataset(df, N_PAIRS):

    N_EQUAL =  N_PAIRS // 2 

    dataset_equal = np.zeros((N_EQUAL, 2 ,100,100,3))
    labels_equal = np.zeros((N_EQUAL,1))

    dataset_diff = np.zeros((N_EQUAL, 2 ,100,100,3))
    labels_diff = np.zeros((N_EQUAL,1))


    # creiamo prima le coppie vere 

    for i in range(20):
        print('lichen: ',i)
        temp_df = df[df['label'] == i]
        number_of_lichen =temp_df.shape[0]
        xtracted_couple = []
        cont = 0

        while cont < N_EQUAL//20:
            im1, im2 = np.random.randint(number_of_lichen, size=2)

            if (im1,im2) not in xtracted_couple and (im2,im1) not in xtracted_couple:
                im1_pth = temp_df.iloc[im1]['filepath']
                im2_pth = temp_df.iloc[im2]['filepath']


                image1 = cv2.imread(im1_pth)
                image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
                image1 = cv2.resize(image1,(100,100))
                #image1 = image1.astype('float32')
                #image1 /= 255.0
        
        
                image2 = cv2.imread(im2_pth)
                image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
                image2 = cv2.resize(image2,(100,100))
                #image2 = image2.astype('float32')
                #image2 /= 255.0

                dataset_equal[i*(N_EQUAL//20) + cont,0,:,:,:] = image1 
                dataset_equal[i*(N_EQUAL//20) + cont,1,:,:,:] = image2
                labels_equal[i*(N_EQUAL//20) + cont] = 1 
                cont = cont + 1
            else:
                continue
  
    # now we must deal with pairs fromdifferent species 

    

    print('START RECORDING DIFFERENT PAIRS')
    N_DIFF = N_PAIRS//2
    i = 0
    while  i < N_DIFF:
        xtracted_couple = []
        sp1 = 0
        sp2 = 0
        while sp1== sp2:
            sp1,sp2 = np.random.randint(20, size=2)
    
        # extract images 

        temp_df1 = df[df['label'] == sp1]
        temp_df2 = df[df['label'] == sp2]

        # extract images
        iml1 = np.random.randint(temp_df1.shape[0], size=1)[0]
        iml2 = np.random.randint(temp_df2.shape[1], size=1)[0]

        if (sp1, iml1 , sp2, iml2) not in xtracted_couple and (sp2,iml2,sp1,iml1)not in xtracted_couple:
            print(i)

            im1_pth = temp_df1.iloc[iml1]['filepath']
            im2_pth = temp_df2.iloc[iml2]['filepath']


            image1 = cv2.imread(im1_pth)
            image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
            image1 = cv2.resize(image1,(100,100))
            #image1 = image1.astype('float32')
            #image1 /= 255.0
        
        
            image2 = cv2.imread(im2_pth)
            image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
            image2 = cv2.resize(image2,(100,100))
            #image2 = image2.astype('float32')
            #image2 /= 255.0

            dataset_diff[i,0,:,:,:] = image1 
            dataset_diff[i,1,:,:,:] = image2 

            labels_diff[i] = 0
            i = i +1 
        else:
            continue
  
    X = np.concatenate([dataset_equal, dataset_diff], axis=0)/255
    Y = np.concatenate([labels_equal, labels_diff], axis=0)
    
    return X,Y

In [ ]:
df = create_lichen_dataframe('../data/train', lichens_dict)
X, Y = create_dataset(df, 6000)

In [ ]:
np.save('training_data.npy',X)
np.save('training_labe.npy',Y)

In [ ]:
plt.imshow(X[0,0,:,:,:])

In [ ]:
plt.imshow(X[0,1,:,:,:])